<a href="https://colab.research.google.com/github/data-fabiosantos/database_spark/blob/main/atp_bancodedados_fabiosantos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUCPR - PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ
# CURSO: BIG DATA E INTELIGÊNCIA ANALÍTICA
# MATÉRIA: BANCO DE DADOS APLICADO À BIG DATA
# PROFESSOR: WELLINGTON RODRIGO MONTEIRO
# ALUNO: FÁBIO HENRIQUE DOS SANTOS



Elabore as seguintes etapas:
1. Mapear cada arquivo como uma tabela dentro do Apache Spark SQL, importando cada um deles.
2. Ajustar os tipos de dado correspondentes entre as colunas dos arquivos e das tabelas.



In [ ]:
import os 
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.7'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.7'

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 38.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3fbb4cdc5e95c91eb73e0264d62b49f78f163f7a5e1f2690f24ccdb9c5e83703
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Banco de dados Aplicado a Big Data").config("spark.some.config.option","some-value").getOrCreate()

In [ ]:
from pyspark import SparkContext
sc = SparkContext

In [ ]:
# Load arquivos
tbl_endereco = spark.read.option("header",True).option("sep",";").csv("address.csv")
tbl_city = spark.read.option("header",True).option("sep",";").csv("city.csv")
tbl_customer = spark.read.option("header",True).option("sep",";").csv("customer.csv")
tbl_film = spark.read.option("header",True).option("sep",";").csv("film.csv")
tbl_language = spark.read.option("header",True).option("sep",";").csv("language.csv")


# Views do SparkSQL
tbl_endereco.createOrReplaceTempView("tbl_endereco")
tbl_city.createOrReplaceTempView("tbl_city")
tbl_customer.createOrReplaceTempView("tbl_customer")
tbl_film.createOrReplaceTempView("tbl_film")
tbl_language.createOrReplaceTempView("tbl_language")

# Analises

In [ ]:
tbl_endereco.columns

['address_id',
 'address',
 'address2',
 'district',
 'city_id',
 'postal_code',
 'phone',
 'location',
 'last_update']

In [ ]:
tbl_endereco

DataFrame[address_id: string, address: string, address2: string, district: string, city_id: string, postal_code: string, phone: string, location: string, last_update: string]

In [ ]:
spark.sql("""select tbl_language.name from tbl_language""").show()

+--------+
|    name|
+--------+
| English|
| Italian|
|Japanese|
|Mandarin|
|  French|
|  German|
+--------+



3. Elaborar uma consulta para responder à seguinte pergunta:
Quantos filmes existem em cada língua/idioma?


In [ ]:
spark.sql("""

select tbl_language.language_id, tbl_language.name, count(*) as filmesEmCadaIdioma
from tbl_language, tbl_film
where tbl_language.language_id = tbl_film.language_id
group by tbl_language.language_id, tbl_language.name
order by filmesEmCadaIdioma asc
limit 1

""").show()

+-----------+-------+------------------+
|language_id|   name|filmesEmCadaIdioma|
+-----------+-------+------------------+
|          1|English|              1000|
+-----------+-------+------------------+



4. Elaborar uma consulta para responder à seguinte pergunta:
Quantos clientes moram na mesma cidade?

In [ ]:
spark.sql("""

select 
    tbl_customer.customer_id , 
    tbl_customer.first_name,  
    tbl_endereco.address_id ,  
    tbl_city.city_id, 
    tbl_city.city, 
    count(*) as ClientesPorCidade
from 
    tbl_customer, 
    tbl_endereco, 
    tbl_city  
where 
    tbl_customer.address_id = tbl_endereco.address_id and tbl_endereco.address_id = tbl_city.city_id
    
group by 
    tbl_customer.customer_id, 
    tbl_customer.first_name, 
    tbl_endereco.address_id,  
    tbl_city.city_id, 
    tbl_city.city
    
order by tbl_customer.customer_id asc
limit 5


""").show()

+-----------+----------+----------+-------+---------+-----------------+
|customer_id|first_name|address_id|city_id|     city|ClientesPorCidade|
+-----------+----------+----------+-------+---------+-----------------+
|          1|      MARY|         5|      5|    Adana|                1|
|         10|   DOROTHY|        14|     14|al-Manama|                1|
|        100|     ROBIN|       104|    104|   Cavite|                1|
|        101|     PEGGY|       105|    105|  Cayenne|                1|
|        102|   CRYSTAL|       106|    106|   Celaya|                1|
+-----------+----------+----------+-------+---------+-----------------+



Fim